In [ ]:
!pip install -q llama-index
!pip install -q openai
!pip install -q transformers
!pip install -q accelerate
!pip -q install llama-index-core
!pip -q install llama-index-llms-openai
!pip -q install llama-index-llms-replicate
!pip -q install llama-index-embeddings-huggingface

In [ ]:
import os

import pandas as pd

df_rag_train = df2.sample(n=100, random_state=42)  # Use a fixed random state for reproducibility

# Create the testing set by dropping the sampled rows from df2
df2_rag_test = df2.drop(df_rag_train.index)

# Now df_rag_train contains 100 random samples, and df2_test contains the rest
df_rag_train.to_csv('df_rag_train.csv', index=False)
df2_rag_test.to_csv('df2_rag_test.csv', index=False)

from llama_index.core.llms import LLM
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import os
# from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''

documents = SimpleDirectoryReader("gdrive/My Drive/limitations_dataset/df_rag_train").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
# reset index
df2_rag_test.reset_index(inplace=True, drop=False)

df2_rag_test['response_string'] = df2_rag_test.apply(lambda row: f"""Abstract: {row['Abstract']}
Introduction: {row['Introduction']}
Methodology: {row['methodology']}
Conclusion: {row['conclusion']}
""", axis=1)

index.storage_context.persist()

from llama_index.core import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context=storage_context)
from llama_index.core import ServiceContext, set_global_service_context
from llama_index.llms import openai
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display
# set global settings config

llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.llm = llm
Settings.chunk_size = 512
service_context = ServiceContext.from_defaults(llm=llm, chunk_size=800, chunk_overlap=20)
index = VectorStoreIndex.from_documents(documents)
system_prompt = '''You are a helpful, respectful, and honest assistant for generating limitations or shortcomings of a research paper.
 Generate limitations or shortcomings for the following passages from a scientific paper.
 I am providing Abstract, Introduction, Methodology, and Conclusion of a scientific paper.'''

import io
import sys

generated_limitations = []

for i in range(len(df)):  # Assuming you need to iterate from 11 to 11, which is effectively just once
    query_engine = index.as_query_engine(
        similarity_top_k=3,
        streaming=True,
    )
    response = query_engine.query(
        system_prompt + ": " + df2_rag_test['response_string'][i]
    )
    limitation_text = ""
    print("response is: ", response.print_response_stream())
    # Redirect stdout to capture print outputs
    old_stdout = sys.stdout  # Memorize the default stdout stream
    sys.stdout = buffer = io.StringIO()

    try:
        response.print_response_stream()
        limitation_text = buffer.getvalue()  # Get whatever was printed to the "fake" stdout
    finally:
        sys.stdout = old_stdout  # Restore stdout. Important to do this early
    print("limitation text: ",limitation_text.strip())
    generated_limitations.append(limitation_text.strip())